## Project 3: Numerical Methods for Differential Equations

## Task 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import eigs
from mpl_toolkits.mplot3d import axes3d

In [ ]:
def toeplitz_like(dim):
    return csr_matrix((diags([1, -2, 1], [-1, 0, 1], shape=(dim, dim))) * (dim + 1)**2)

def euler_step(A: np.array, y_n: np.array, h: float, alpha, beta):
    return np.concatenate(([alpha], y_n + h * A @ y_n.T, [beta]))

## Task 1.1

In [ ]:
def euler_solve(g_0, N, M, t_end, alpha, beta):
    xx = np.linspace(0, 1, N+2)
    tt = np.linspace(0, t_end, M+1)
    Y = np.zeros((M+1, N+2))
    Y[0,:] = g_0(xx)
    toep = toeplitz_like(N)
    for i in range(M):
        Y[i+1,:] = euler_step(toep, Y[i,:][1:-1], t_end/M, alpha, beta)
    return Y, tt, xx

In [ ]:
def plot3d(Y, Xgrid, tgrid):
     T, X = np.meshgrid(np.transpose(tgrid), Xgrid)

     fig = plt.figure(figsize=(10, 10))
     ax = fig.add_subplot(111, projection='3d')
     ax.plot_surface(T, X, np.transpose(Y))

     ax.set_xlabel('Time')
     ax.set_ylabel('Space')
     ax.set_zlabel('Value')
     ax.set_title('Approximation of Solutions to the Diffusion Equation \n with initial value'+
                                    r'$u(0,x)=x(1-x)^2$')
     
     plt.show()

## Test Task 1.1

In [ ]:
N = 25
M = 2*51**2
t_end = 1
alpha = 0
beta = 0

Y, t_grid, x_grid = euler_solve(lambda x : (x*(1-x))**2, N, M, t_end, alpha, beta)
plot3d(Y, x_grid, t_grid)

##  Experimental Determination of CFL condition

Below, a series of approximation are done for different values of the CFL constant $\mu=\frac{\Delta t}{\Delta x^2}$ to determine for which values the explicit Euler method converges.

In [ ]:
def test_conv(mu, values, convergence):
    N = 25
    M = int(((N+1)**2)/mu)
    t_end = 1
    alpha = 0
    beta = 0
    Y, t_grid, x_grid = euler_solve(lambda x : (x*(1-x))**2, N, M, t_end, alpha, beta)
    
    if(np.sum(abs(Y[-1,:]))<0.001):
        values.append(f"CFL={mu}")
        convergence.append("Converges")
    else:
        values.append(f"CFL={mu}")
        convergence.append("Diverges")


In [ ]:
values = []
convergence = []

for k in range(10):
    mu = (k+1) / 10 
    test_conv(mu, values, convergence)
    
df = pd.DataFrame({
    "values": values,
    "convergence": convergence,
    })

df.style \
.format(precision=3, thousands=".", decimal=",") \
.format_index(str.upper, axis=1)
display(df)

From the above broad testing, we can determine that the line of convergence is in the intervall $\mu\in[5,6)$.

In [ ]:
testcases = [num/100 for num in range(50, 60)]
for test in testcases:
    test_conv(test)

df = pd.DataFrame({
    "values": values,
    "convergence": convergence,
    })

df.style \
.format(precision=3, thousands=".", decimal=",") \
.format_index(str.upper, axis=1)
display(df)

The inteval is now narrowed down to $\mu\in[0.5,0.51)$ and further experiments are conducted

In [ ]:
def conv(mu):
    N = 25
    M = int(((N+1)**2)/mu)
    t_end = 1
    alpha = 0
    beta = 0
    Y, t_grid, x_grid = euler_solve(lambda x : (x*(1-x))**2, N, M, t_end, alpha, beta)
    if(np.sum(abs(Y[-1,:]))<0.00005):
        return True
    else:
        return False
    
mu = 0.505
while(not conv(mu)):
    mu -= (mu - 0.5)/2
print(mu)

With a tolerance of $0.00005$, the first value of convergence is $\mu = 0.50125$ and for smaller values, the code takes too long to run without further optimization which leaves the experimenta result och the maximal CFL condition in the interval $\mu\in[0.5, 0.50125)$ which is very close to the theoretical value of $\frac{1}{2}$.

Visualisation of divirging solution:

In [ ]:
mu = 0.503
N = 25
M = int(((N+1)**2)/mu)
t_end = 1
alpha = 0
beta = 0
Y, t_grid, x_grid = euler_solve(lambda x : (x*(1-x))**2, N, M, t_end, alpha, beta)
plot3d(Y, x_grid, t_grid)

## Task 1.2

\begin{equation*}
    u^{n+1}=u^n+\frac{1}{2}\Delta t\left(T_{\Delta x}u^n+T_{\Delta x}u^{n+1}\right)
\end{equation*}

\begin{equation*}
    u^{n+1}=\left(I-\frac{\Delta t}{2}T_{\Delta x}\right)^{-1}\left(I+\frac{\Delta t}{2}T_{\Delta x}\right)u^n
\end{equation*}

In [ ]:
def Crank_Nicholson_step(Tdx, uold, dt, alpha, beta):
    I = identity(Tdx.get_shape()[0])
    RHS = I-0.5*dt*Tdx
    LHS = (I+0.5*dt*Tdx) @ (uold.T)
    return np.concatenate(([alpha], spsolve(RHS, LHS), [beta]))
        
def Crank_Nicholson_solver(g_0, N, M, t_end, alpha, beta):
    xx = np.linspace(0, 1, N+2)
    tt = np.linspace(0, t_end, M+1)
    Y = np.zeros((M+1, N+2))
    Y[0,:] = g_0(xx)
    toep = toeplitz_like(N)
    for i in range(M):
        Y[i+1,:] = Crank_Nicholson_step(toep, Y[i,:][1:-1], t_end/M, alpha, beta)
    return Y, tt, xx

In [ ]:
N = 50
M = 50
t_end = 1
alpha = 0
beta = 0
Y, t_grid, x_grid = Crank_Nicholson_solver(lambda x : (x*(1-x))**2, N, M, t_end, alpha, beta)
plot3d(Y, x_grid, t_grid)

# Part 2

\begin{equation*}
    \begin{cases}
        u_t = au_x\\
        u(t,0)=u(t,1), \forall t\\
        u(0,x)=g(x)
    \end{cases}
\end{equation*}
$\Delta x = \frac{1}{N}$, $N$ evaluation points, $x_1=0, x_{N+1}=1$ and $u_{N+1}=u_1$. $g(0)=g(1)$ and $g'(0)=g'(1)$

Lax-Wendroff scheme:
\begin{equation*}
    \begin{cases}
        u_j^{n+1}=\frac{a\mu}{2}(1+a\mu)_{j-1}^n+(1-a^2\mu^2)u_j^n+\frac{a\mu}{2}(a\mu-1)u_{j+1}^n\\
        \mu=\frac{\Delta t}{\Delta x}
    \end{cases}
\end{equation*}

In [ ]:
def LaxWen_Toeplitz(amu, dim):
     A = diags([amu/2*(amu+1), 1-amu**2, amu/2*(amu-1)], [-1, 0, 1], shape=(dim, dim))
     A[-1,0] = amu/2*(amu-1)
     A[0,-1] = amu/2*(amu+1)
     return csr_matrix(A)

def LaxWen_step(u_old, A):
     return A @ np.transpose(u_old)

def LaxWen_solver(g0, t_end, N, M, a):
     return 

In [ ]:
def pulse(x):
    return np.exp(-100*(x-0.5)**2)